**Step 1:**

I have got pre-processed fastq files (guaterria and non-guaterria plastomes) assembled to uvaria afzelii with annotations (incl. from dogma). I preprocess the files into fasta format in Python and save them to the same folder as the original. 

For Guatteria: D:\Annonaceae plastome data\Annonaceae plastome data\Plastome data\Guatteria

In [ ]:
import glob
from Bio import SeqIO
import os

input_folder_guatteria = r"D:\Annonaceae plastome data\Annonaceae plastome data\Plastome data\Guatteria"

# Find all FASTQ files in that folder
fastq_files_guatteria = glob.glob(os.path.join(input_folder_guatteria, "*.fastq"))

for fastq_file_guatteria in fastq_files_guatteria:
    output_file_guatteria = os.path.splitext(fastq_file_guatteria)[0] + ".fasta"
    with open(output_file_guatteria, "w") as out_handle:
        count = SeqIO.convert(fastq_file_guatteria, "fastq", out_handle, "fasta")
    #print(f"Converted {count} records from {fastq_file} to {output_file}")


For non-Guatteria:

In [4]:
input_folder_non_guatteria = r"D:\Annonaceae plastome data\Annonaceae plastome data\Plastome data\non-Guatteria"

# Find all FASTQ files in that folder
fastq_files_non_guatteria = glob.glob(os.path.join(input_folder_non_guatteria, "*.fastq"))

for fastq_file_non_guatteria in fastq_files_non_guatteria:
    output_file_non_guatteria = os.path.splitext(fastq_file_non_guatteria)[0] + ".fasta"
    with open(output_file_non_guatteria, "w") as out_handle:
        count = SeqIO.convert(fastq_file_non_guatteria, "fastq", out_handle, "fasta")
    #print(f"Converted {count} records from {fastq_file} to {output_file}")

**Step 2:**

Prepare Input Files (FASTA) - may want to combine all FASTA sequences into one file:

In [5]:
input_files_mafft = glob.glob(os.path.join(input_folder_guatteria, input_folder_non_guatteria, "*.fasta"))
  # etc.
combined_file_mafft = "all_plastomes.fasta"

with open(combined_file_mafft, "w") as outfile:
    for fname in input_files_mafft:
        for record in SeqIO.parse(fname, "fasta"):
            SeqIO.write(record, outfile, "fasta")


**Step 3:**

Run MAFFT from Python

In [ ]:
import subprocess

input_fasta_mafft = "all_plastomes.fasta"
output_fasta_mafft = "aligned_plastomes.fasta"

# Adjust `--auto` for best general results, or use `--globalpair --maxiterate 1000` for accuracy
mafft_path = r"D:\mafft\mafft-win\mafft.bat"  # Use raw string (r"...")

cmd = [mafft_path, "--auto", input_fasta_mafft]

with open(output_fasta_mafft, "w") as out_f:
    subprocess.run(cmd, stdout=out_f)

print("Alignment completed with MAFFT.")